In [6]:
# !pip3 install -U openai-whisper -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install noisereduce -i https://mirrors.aliyun.com/pypi/simple/ 

In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from tqdm import tqdm
import torch
import librosa
import numpy as np
import glob
import os

import sys
sys.path.append("/home/shoinoue/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

import whisper
from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer
import pandas as pd
import scipy.stats as st
import math
from IPython.display import clear_output

sr = 16000

In [2]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
spk = "ASI"

## Whisper

In [3]:
models = {
    # "tiny": whisper.load_model("tiny.en"),
    # "base": whisper.load_model("base.en"),
    # "small": whisper.load_model("small.en"),
    "medium": whisper.load_model("medium.en"),
    # "large": whisper.load_model("large"),
}

In [6]:
speakers = ["ASI", "RRBI", "SVBI", "TNI"]

In [51]:
import shutil

In [64]:
for spk in speakers:
    print(spk)
    text_dir = "./SPAT/transliteration/"
    base_dir = f"/mntcephfs/data/audiow/shoinoue/Dataset/CosyVoice/{spk}_allsamples/English/wav/"
    save_dir = f"/mntcephfs/data/audiow/shoinoue/Dataset/CosyVoice/{spk}/English/wav/"
    os.makedirs(save_dir, exist_ok=True)
    basenames = list(set([os.path.basename(a).split(".")[0][:-2] for a in glob.glob(base_dir+f"*.wav")]))
    basenames.sort()
    for basename in tqdm(basenames):
        textpath = text_dir + basename + ".npy"
        sentences = np.load(textpath, allow_pickle=True).item()
        gttext = normalizer(sentences["English"])
        audiofiles = glob.glob(base_dir+f"{basename}*.wav")
        audiofiles.sort()
        wers = []
        lens = []
        for path in audiofiles:
            predtext = normalizer(models["medium"].transcribe(path)["text"])
            wer = jiwer.wer([gttext], [predtext])
            wers += [wer]
            lens += [len(librosa.load(path)[0])]
        idx = np.argmin(np.array(wers))
        print(wers, idx)
        src = audiofiles[idx]
        trg = save_dir + basename + f".wav"
        shutil.copy(src, trg)
    break

ASI


  4%|▎         | 1/27 [00:01<00:28,  1.11s/it]

[0.125, 0.125] 0


  7%|▋         | 2/27 [00:01<00:24,  1.02it/s]

[0.125, 0.375] 0


...

[0.0, 0.0] 0


 96%|█████████▋| 26/27 [00:25<00:01,  1.01s/it]

[0.0, 0.0] 0


100%|██████████| 27/27 [00:26<00:00,  1.00it/s]

[0.0, 0.0] 0


In [61]:
basenames

['arctic_a0013']

In [58]:
basenames

[]

In [50]:
trg

'/mntcephfs/lab_data/shoinoue/Dataset/CosyVoice/ASI/English/wav/arctic_a0013.wav'

'/mntcephfs/lab_data/shoinoue/Dataset/CosyVoice/ASI_allsamples/English/wav/arctic_a0013-0.wav'

In [44]:
wer

0.0

In [41]:
predtext

'he was a head shorter than his companion of almost delicate physique'

In [42]:
gttext

'he was a head shorter than his companion of almost delicate physique'

In [10]:
files = glob.glob(base_dir + f"/transcript/*")
files.sort()
groundtruth_dir = {}
for file in files:
    f = open(file, "r")
    text = f.read()
    key = os.path.basename(file)[:-4]
    groundtruth_dir[key] = normalizer(text)

NameError: name 'base_dir' is not defined

In [20]:
size = "medium"

In [43]:
files = glob.glob("test/korean/arctic*.wav")
# files = glob.glob("test/gt/arctic*.wav")
# files = glob.glob("test/eng/arctic*.wav")
files.sort()

In [42]:
# Ground truth
for path in files:
    basename = os.path.basename(path)[:-4]
    predtext = normalizer(models[size].transcribe(path)["text"])
    wer = jiwer.wer(groundtruth_dir[basename], [predtext])
    # wer = jiwer.cer(groundtruth_dir[basename], [predtext])
    print(wer, predtext)

0.125 author of the danger trail philip still etc
0.25 not at this particular case tom apologized with more
0.0 for the 20th time that evening the 2 men shook hands
0.4 loads but i am glad to see your game feel
0.0 will we ever forget it


In [39]:
# English
for path in files:
    basename = os.path.basename(path)[:-4]
    predtext = normalizer(models[size].transcribe(path)["text"])
    wer = jiwer.wer(groundtruth_dir[basename], [predtext])
    # wer = jiwer.cer(groundtruth_dir[basename], [predtext])
    print(wer, predtext)

0.0 author of the danger trail philip steels etc
0.0 not at this particular case tom apologized whittemore
0.0 for the 20th time that evening the 2 men shook hands
0.0 lord but i am glad to see you again phil
0.0 will we ever forget it


In [44]:
# Korean accent
for path in files:
    basename = os.path.basename(path)[:-4]
    predtext = normalizer(models[size].transcribe(path)["text"])
    wer = jiwer.wer(groundtruth_dir[basename], [predtext])
    # wer = jiwer.cer(groundtruth_dir[basename], [predtext])
    print(wer, predtext)

0.625 also the outdoor danger trail phillips tills etc
0.625 note that this particular case time apologized with more
0.0 for the 20th time that evening the 2 men shook hands
0.3 rodopod i am glad to see you again pill
0.8 well we are billpuggises


In [ ]:
groundtrut

In [24]:
predtext

'also the outdoor danger trail phillips tills etc'

In [25]:
wer

1.0

In [65]:
save = True

result = {}
result["predtext"] = {}
result["wer"] = {}
audiofiles = glob.glob(base_dir + f"{spk}/wav/*.wav")
audiofiles.sort()
for size in models:
    print(size)
    model = models[size]
    result["predtext"][size] = {}
    result["wer"][size] = {}
    for file in tqdm(audiofiles):
        key = os.path.basename(file)[:-4]
        predtext = normalizer(models[size].transcribe(file)["text"])
        result["predtext"][size][key] = predtext
        wer = jiwer.wer(groundtruth_dir[key], [predtext])
        result["wer"][size][key] = wer
            
savefile = "./wer_whisper_result.npy"
if save:
    np.save(savefile, result)

tiny


100%|██████████| 1131/1131 [03:59<00:00,  4.73it/s]


base


100%|██████████| 1131/1131 [04:34<00:00,  4.11it/s]


small


100%|██████████| 1131/1131 [06:06<00:00,  3.09it/s]


medium


100%|██████████| 1131/1131 [10:10<00:00,  1.85it/s]


# Evaluation

In [151]:
data = np.load("./wer_whisper_result.npy", allow_pickle=True).item()
facebook = np.load("./wer_facebook_result.npy", allow_pickle=True).item()
data["predtext"]["facebook"] = facebook["predtext"]["facebook"]
data["wer"]["facebook"] = facebook["wer"]["facebook"]
# speechbrain = np.load("./wer_speechbrain_result.npy", allow_pickle=True).item()
# data["predtext"]["speechbrain"] = speechbrain["predtext"]["speechbrain"]
# data["wer"]["speechbrain"] = speechbrain["wer"]["speechbrain"]
basenames = np.array(list(data["predtext"][targets[0]].keys()))

In [156]:
targets = ["tiny", "base", "small", "medium"]
# targets = ["facebook"]

In [157]:
wers = np.array([np.array(list(data["wer"][size].values())) for size in targets])
wers = np.mean(wers, axis=0)

In [155]:
sample_num = 5 

order = np.argsort(wers)
print("Lower WER")
for i, idx in enumerate(order[:sample_num]):
    basename = basenames[idx]
    wer = wers[idx]
    path = base_dir + f"{spk}/wav/{basename}.wav"
    print(basename, wer)
    play_audio(path, sr)
    
for i, idx in enumerate(order[-sample_num:]):
    basename = basenames[idx]
    wer = wers[idx]
    path = base_dir + f"{spk}/wav/{basename}.wav"
    print(basename, wer)
    play_audio(path, sr)

Lower WER
arctic_b0032 0.0


arctic_b0327 0.0


arctic_b0332 0.0


arctic_b0335 0.0


arctic_a0322 0.0


arctic_b0128 0.5499999999999999


arctic_a0421 0.55


arctic_a0589 0.6


arctic_b0257 0.6666666666666667


arctic_a0579 0.8999999999999999


In [160]:
basenames[wers==0]

array(['arctic_a0004', 'arctic_a0005', 'arctic_a0009', 'arctic_a0010',
       'arctic_a0011', 'arctic_a0013', 'arctic_a0014', 'arctic_a0015',
       'arctic_a0018', 'arctic_a0023', 'arctic_a0025', 'arctic_a0026',
       'arctic_a0027', 'arctic_a0028', 'arctic_a0030', 'arctic_a0031',
       'arctic_a0032', 'arctic_a0033', 'arctic_a0035', 'arctic_a0036',
       'arctic_a0037', 'arctic_a0038', 'arctic_a0039', 'arctic_a0040',
       'arctic_a0042', 'arctic_a0045', 'arctic_a0046', 'arctic_a0047',
       'arctic_a0050', 'arctic_a0051', 'arctic_a0052', 'arctic_a0053',
       'arctic_a0054', 'arctic_a0057', 'arctic_a0058', 'arctic_a0059',
       'arctic_a0060', 'arctic_a0062', 'arctic_a0067', 'arctic_a0070',
       'arctic_a0071', 'arctic_a0075', 'arctic_a0081', 'arctic_a0087',
       'arctic_a0093', 'arctic_a0094', 'arctic_a0095', 'arctic_a0096',
       'arctic_a0100', 'arctic_a0101', 'arctic_a0102', 'arctic_a0104',
       'arctic_a0105', 'arctic_a0107', 'arctic_a0109', 'arctic_a0112',
      